# Brazilian E-commerce Dataset - Initial Data Exploration

This notebook provides a comprehensive exploration of the Brazilian E-commerce dataset, including:
- Data loading and validation
- Data quality assessment
- Basic statistical analysis
- Relationship analysis between datasets
- Initial findings and insights

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os

# Add parent directory to path to import our data_loader module
sys.path.append('..')
from data_loader import DataLoader, load_brazilian_ecommerce_data

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Data Loading and Initial Overview

In [ ]:
# Load all datasets using our data loader
print("Loading Brazilian E-commerce datasets...")
datasets, summary = load_brazilian_ecommerce_data()

print(f"\nSuccessfully loaded {len(datasets)} datasets")
print("\nDataset Summary:")
display(summary)

In [ ]:
# Display basic information about each dataset
for name, df in datasets.items():
    print(f"\n{'='*50}")
    print(f"DATASET: {name.upper()}")
    print(f"{'='*50}")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nData Types:")
    print(df.dtypes)
    print(f"\nFirst 3 rows:")
    display(df.head(3))

## 2. Data Quality Assessment

In [ ]:
# Comprehensive missing value analysis
def analyze_missing_values(datasets):
    missing_analysis = []
    
    for name, df in datasets.items():
        missing_counts = df.isnull().sum()
        missing_percentages = (missing_counts / len(df)) * 100
        
        for column in df.columns:
            if missing_counts[column] > 0:
                missing_analysis.append({
                    'Dataset': name,
                    'Column': column,
                    'Missing_Count': missing_counts[column],
                    'Missing_Percentage': round(missing_percentages[column], 2),
                    'Total_Rows': len(df)
                })
    
    return pd.DataFrame(missing_analysis).sort_values('Missing_Percentage', ascending=False)

missing_df = analyze_missing_values(datasets)
print("Missing Values Analysis:")
display(missing_df)

In [ ]:
# Visualize missing values
if not missing_df.empty:
    plt.figure(figsize=(14, 8))
    
    # Create a bar plot of missing percentages
    missing_plot_data = missing_df.head(15)  # Top 15 columns with missing values
    
    bars = plt.bar(range(len(missing_plot_data)), missing_plot_data['Missing_Percentage'])
    plt.xlabel('Dataset.Column')
    plt.ylabel('Missing Percentage (%)')
    plt.title('Top 15 Columns with Missing Values')
    plt.xticks(range(len(missing_plot_data)), 
               [f"{row['Dataset']}.{row['Column']}" for _, row in missing_plot_data.iterrows()], 
               rotation=45, ha='right')
    
    # Add value labels on bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found in any dataset!")

In [ ]:
# Duplicate analysis
def analyze_duplicates(datasets):
    duplicate_analysis = []
    
    for name, df in datasets.items():
        total_rows = len(df)
        duplicate_rows = df.duplicated().sum()
        duplicate_percentage = (duplicate_rows / total_rows) * 100 if total_rows > 0 else 0
        
        duplicate_analysis.append({
            'Dataset': name,
            'Total_Rows': total_rows,
            'Duplicate_Rows': duplicate_rows,
            'Duplicate_Percentage': round(duplicate_percentage, 2),
            'Unique_Rows': total_rows - duplicate_rows
        })
    
    return pd.DataFrame(duplicate_analysis).sort_values('Duplicate_Percentage', ascending=False)

duplicates_df = analyze_duplicates(datasets)
print("Duplicate Analysis:")
display(duplicates_df)

## 3. Dataset Relationships and Foreign Key Analysis

In [ ]:
# Analyze relationships between datasets
loader = DataLoader()
loader.datasets = datasets
relationships = loader.validate_data_relationships()

print("Dataset Relationship Analysis:")
for relationship, details in relationships.items():
    print(f"\n{relationship.upper().replace('_', ' ')}:")
    for key, value in details.items():
        print(f"  {key.replace('_', ' ').title()}: {value:,}")

In [ ]:
# Analyze unique identifiers in each dataset
def analyze_unique_identifiers(datasets):
    identifier_analysis = []
    
    # Define potential identifier columns for each dataset
    identifier_columns = {
        'customers': ['customer_id', 'customer_unique_id'],
        'orders': ['order_id'],
        'order_items': ['order_id', 'order_item_id'],
        'order_payments': ['order_id'],
        'order_reviews': ['review_id', 'order_id'],
        'products': ['product_id'],
        'sellers': ['seller_id'],
        'geolocation': ['geolocation_zip_code_prefix'],
        'product_categories': ['product_category_name']
    }
    
    for dataset_name, df in datasets.items():
        if dataset_name in identifier_columns:
            for col in identifier_columns[dataset_name]:
                if col in df.columns:
                    total_rows = len(df)
                    unique_values = df[col].nunique()
                    null_values = df[col].isnull().sum()
                    
                    identifier_analysis.append({
                        'Dataset': dataset_name,
                        'Column': col,
                        'Total_Rows': total_rows,
                        'Unique_Values': unique_values,
                        'Null_Values': null_values,
                        'Uniqueness_Ratio': round(unique_values / (total_rows - null_values), 4) if (total_rows - null_values) > 0 else 0
                    })
    
    return pd.DataFrame(identifier_analysis)

identifiers_df = analyze_unique_identifiers(datasets)
print("Unique Identifier Analysis:")
display(identifiers_df)

## 4. Basic Statistical Analysis

In [ ]:
# Analyze numerical columns across all datasets
def analyze_numerical_columns(datasets):
    numerical_analysis = []
    
    for name, df in datasets.items():
        numerical_cols = df.select_dtypes(include=[np.number]).columns
        
        for col in numerical_cols:
            stats = df[col].describe()
            numerical_analysis.append({
                'Dataset': name,
                'Column': col,
                'Count': int(stats['count']),
                'Mean': round(stats['mean'], 2),
                'Std': round(stats['std'], 2),
                'Min': stats['min'],
                'Max': stats['max'],
                'Median': round(stats['50%'], 2)
            })
    
    return pd.DataFrame(numerical_analysis)

numerical_df = analyze_numerical_columns(datasets)
print("Numerical Columns Analysis:")
display(numerical_df)

In [ ]:
# Analyze categorical columns
def analyze_categorical_columns(datasets, max_unique_values=20):
    categorical_analysis = []
    
    for name, df in datasets.items():
        categorical_cols = df.select_dtypes(include=['object']).columns
        
        for col in categorical_cols:
            unique_count = df[col].nunique()
            if unique_count <= max_unique_values:  # Only analyze columns with reasonable number of categories
                top_values = df[col].value_counts().head(5)
                categorical_analysis.append({
                    'Dataset': name,
                    'Column': col,
                    'Unique_Values': unique_count,
                    'Most_Common': top_values.index[0] if len(top_values) > 0 else 'N/A',
                    'Most_Common_Count': top_values.iloc[0] if len(top_values) > 0 else 0,
                    'Top_5_Values': dict(top_values)
                })
    
    return pd.DataFrame(categorical_analysis)

categorical_df = analyze_categorical_columns(datasets)
print("Categorical Columns Analysis (with ≤20 unique values):")
for _, row in categorical_df.iterrows():
    print(f"\n{row['Dataset']}.{row['Column']}:")
    print(f"  Unique values: {row['Unique_Values']}")
    print(f"  Most common: {row['Most_Common']} ({row['Most_Common_Count']} occurrences)")
    print(f"  Top 5 values: {row['Top_5_Values']}")

## 5. Date/Time Analysis

In [ ]:
# Identify and analyze date columns
def analyze_date_columns(datasets):
    date_analysis = []
    
    for name, df in datasets.items():
        # Look for columns that might contain dates
        potential_date_cols = [col for col in df.columns if any(keyword in col.lower() 
                              for keyword in ['date', 'time', 'timestamp'])]
        
        for col in potential_date_cols:
            # Try to convert to datetime and analyze
            try:
                # Sample a few values to check format
                sample_values = df[col].dropna().head(10).tolist()
                null_count = df[col].isnull().sum()
                null_percentage = (null_count / len(df)) * 100
                
                date_analysis.append({
                    'Dataset': name,
                    'Column': col,
                    'Null_Count': null_count,
                    'Null_Percentage': round(null_percentage, 2),
                    'Sample_Values': sample_values[:3],  # First 3 non-null values
                    'Data_Type': str(df[col].dtype)
                })
            except Exception as e:
                print(f"Error analyzing {name}.{col}: {str(e)}")
    
    return pd.DataFrame(date_analysis)

date_df = analyze_date_columns(datasets)
print("Date/Time Columns Analysis:")
display(date_df)

## 6. Key Findings and Initial Insights

In [ ]:
# Generate comprehensive data quality report
def generate_data_quality_report(datasets, missing_df, duplicates_df, relationships):
    report = []
    report.append("=" * 80)
    report.append("BRAZILIAN E-COMMERCE DATASET - DATA QUALITY REPORT")
    report.append("=" * 80)
    
    # Overall statistics
    total_rows = sum(df.shape[0] for df in datasets.values())
    total_columns = sum(df.shape[1] for df in datasets.values())
    total_memory = sum(df.memory_usage(deep=True).sum() for df in datasets.values()) / 1024 / 1024
    
    report.append(f"\n📊 OVERALL STATISTICS:")
    report.append(f"   • Total datasets: {len(datasets)}")
    report.append(f"   • Total rows: {total_rows:,}")
    report.append(f"   • Total columns: {total_columns}")
    report.append(f"   • Total memory usage: {total_memory:.2f} MB")
    
    # Data quality issues
    report.append(f"\n🔍 DATA QUALITY ISSUES:")
    
    # Missing values
    if not missing_df.empty:
        high_missing = missing_df[missing_df['Missing_Percentage'] > 10]
        report.append(f"   • Columns with >10% missing values: {len(high_missing)}")
        if len(high_missing) > 0:
            report.append(f"     - Highest: {high_missing.iloc[0]['Dataset']}.{high_missing.iloc[0]['Column']} ({high_missing.iloc[0]['Missing_Percentage']:.1f}%)")
    else:
        report.append(f"   • No missing values found!")
    
    # Duplicates
    high_duplicates = duplicates_df[duplicates_df['Duplicate_Percentage'] > 5]
    report.append(f"   • Datasets with >5% duplicates: {len(high_duplicates)}")
    if len(high_duplicates) > 0:
        report.append(f"     - Highest: {high_duplicates.iloc[0]['Dataset']} ({high_duplicates.iloc[0]['Duplicate_Percentage']:.1f}%)")
    
    # Relationship issues
    report.append(f"\n🔗 RELATIONSHIP ANALYSIS:")
    for rel_name, rel_data in relationships.items():
        report.append(f"   • {rel_name.replace('_', ' ').title()}:")
        for key, value in rel_data.items():
            report.append(f"     - {key.replace('_', ' ').title()}: {value:,}")
    
    # Recommendations
    report.append(f"\n💡 RECOMMENDATIONS:")
    report.append(f"   1. Address high missing value columns (especially review comments)")
    report.append(f"   2. Remove duplicate records from geolocation dataset")
    report.append(f"   3. Investigate foreign key mismatches")
    report.append(f"   4. Convert date columns to proper datetime format")
    report.append(f"   5. Create derived features for business analysis")
    
    return "\n".join(report)

# Generate and display the report
quality_report = generate_data_quality_report(datasets, missing_df, duplicates_df, relationships)
print(quality_report)

In [ ]:
# Save the data quality report to a file
with open('../reports/data_quality_report.txt', 'w', encoding='utf-8') as f:
    f.write(quality_report)

print("\n✅ Data quality report saved to 'reports/data_quality_report.txt'")
print("\n🎯 NEXT STEPS:")
print("   1. Proceed with data cleaning based on identified issues")
print("   2. Handle missing values using appropriate strategies")
print("   3. Remove duplicates and resolve foreign key issues")
print("   4. Convert date columns and create derived features")
print("   5. Begin business analysis once data is clean")